This Notebook was used to generate boxplots of all features for the DoS and DDoS attack classes

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
from os import cpu_count
from math import floor
import pandas as pd
import numpy as np

In [6]:
datasets={}
for dirname, _, filenames in os.walk('/project_ghent/raman/netflow_datasets'):
    for filename in filenames:
        if filename.endswith("parquet"):
            datasets[filename.replace(".parquet","")]=pd.read_parquet(os.path.join(dirname, filename))

In [7]:
for k,v in datasets.items():
    print(k)
    print(v.Attack.unique())

NF-ToN-IoT-V2
['ransomware' 'Benign' 'xss' 'scanning' 'password' 'dos' 'ddos'
 'injection' 'mitm' 'backdoor']
NF-UNSW-NB15-V2
['Benign' 'Exploits' 'Generic' 'Fuzzers' 'Backdoor' 'DoS' 'Reconnaissance'
 'Shellcode' 'Worms' 'Analysis']
NF-BoT-IoT-V2
['DoS' 'DDoS' 'Reconnaissance' 'Benign' 'Theft']
NF-CSE-CIC-IDS2018-V2
['SSH-Bruteforce' 'Benign' 'DDoS attacks-LOIC-HTTP' 'DDOS attack-HOIC'
 'DoS attacks-Slowloris' 'DoS attacks-Hulk' 'FTP-BruteForce'
 'Infilteration' 'Bot' 'DoS attacks-GoldenEye' 'Brute Force -Web'
 'DoS attacks-SlowHTTPTest' 'SQL Injection' 'DDOS attack-LOIC-UDP'
 'Brute Force -XSS']


In [8]:
for ds,data in datasets.items():
    ddos=data[data['Attack'].str.contains("ddos",case=False)]
    ddos.Attack="DDoS"#uniform name
    dos=data[data['Attack'].str.contains("dos",case=False)]
    dos.Attack="DoS"#uniform name
    dos.drop(index=ddos.index,inplace=True)
    benign=data[data['Attack'] == 'Benign']
    
    data = pd.concat(objs=[ddos, dos, benign])
    
    # random forest classifier works internally with float32 so values too big will give errors
    data = data[data['SRC_TO_DST_SECOND_BYTES'] <= np.finfo(np.float32).max]
    data = data[data['DST_TO_SRC_SECOND_BYTES'] <= np.finfo(np.float32).max]
    
    data=data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT']) # dropping metadata

    
    datasets[ds]=data

/tmp/ipykernel_184/3237961987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddos.Attack="DDoS"#uniform name
/tmp/ipykernel_184/3237961987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dos.Attack="DoS"#uniform name
/tmp/ipykernel_184/3237961987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dos.drop(index=dd

In [9]:
for k,v in datasets.items():
    print(k)
    print(v.Attack.value_counts())

NF-ToN-IoT-V2
Attack
Benign    3601194
DDoS      1746590
DoS        654359
Name: count, dtype: int64
NF-UNSW-NB15-V2
Attack
Benign    1911666
DoS          4172
Name: count, dtype: int64
NF-BoT-IoT-V2
Attack
DDoS      14280259
DoS       13645057
Benign      129437
Name: count, dtype: int64
NF-CSE-CIC-IDS2018-V2
Attack
Benign    15101526
DDoS       1276019
DoS         483999
Name: count, dtype: int64


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for feature in data.columns[:-2]:
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 5))

    # Plot the DoS boxplot on the first subplot
    boxplot_dict1 = ax1.boxplot([value[value.Attack=="DoS"][feature] for value in datasets.values()])

    # Set the title, xlabel, and ylabel for the first subplot
    ax1.set_title(f"dos: {feature}")
    ax1.set_xlabel("dataset")
    ax1.set_ylabel(feature)

    # Set the x-axis tick labels for the first subplot
    ax1.set_xticklabels(datasets.keys())

    # Plot the DDoS boxplot on the second subplot
    boxplot_dict2 = ax2.boxplot([value[value.Attack=="DDoS"][feature] for value in datasets.values()])

    # Set the title, xlabel, and ylabel for the second subplot
    ax2.set_title(f"ddos: {feature}")
    ax2.set_xlabel("dataset")
    ax2.set_ylabel(feature)

    # Set the x-axis tick labels for the second subplot
    ax2.set_xticklabels(datasets.keys())

    # Display the plot
    plt.show()
    fig.savefig(f"./plots/{feature}.png")